In [1]:
import tqdm, json, os, re, operator
import string, pprint, itertools, seaborn, collections, numpy as np, csv, requests, nltk, random
import JudgmentsProcessor
import JudgmentsClassifier
import pickle

In [2]:
judgments = []
signatures = []


def read_file(file_path):
   
    print ("Processing file" + file_path)
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in tqdm.tqdm(json_content):
            judgment = []
            if item['courtType'] in ['COMMON', 'SUPREME']:
                courtCases = item['courtCases']
                signatures_for_judgment = []
                for courtCase in courtCases:
                    signature = courtCase['caseNumber']
                    signatures_for_judgment.append(signature)
                signatures.append(signatures_for_judgment)
                item_count += 1
                text_content = re.sub("<.*?>", "", item["textContent"])
                text_content = text_content.replace('-\n', '')
                word_content = text_content.split()
                topicSpecificPunctuation = '„”–§…«»'
                translator = str.maketrans('', '', string.punctuation+topicSpecificPunctuation)

                for word in word_content:
                    word = word.translate(translator).lower()
                    if len(word)>0:
                        judgment.append(word)
            
                judgments.append(judgment)

In [3]:
def read_all_judgments_from_2018():
        for filename in os.listdir("../data_filtered_2/"):
            read_file("../data_filtered_2/" + filename)

In [4]:
read_all_judgments_from_2018()

 47%|████▋     | 47/100 [00:00<00:00, 431.03it/s]

Processing file../data_filtered_2/judgments-3163.json
Processing file../data_filtered_2/judgments-3168.json


 26%|██▌       | 26/100 [00:00<00:00, 252.40it/s]

Processing file../data_filtered_2/judgments-3164.json


 40%|████      | 40/100 [00:00<00:00, 395.63it/s]

...

 37%|███▋      | 37/100 [00:00<00:00, 350.15it/s]

Processing file../data_filtered_2/judgments-3166.json


 31%|███       | 31/100 [00:00<00:00, 277.67it/s]

Processing file../data_filtered_2/judgments-3170.json


100%|██████████| 100/100 [00:00<00:00, 284.22it/s]


In [5]:
signatures[0]

['VI Ca 651/17']

In [9]:
test_idx = random.sample(range(0, 783), int(195))
train_idx = list(range(0, 783))
train_idx = [el for el in train_idx if el not in test_idx]

In [5]:
judge_processor = JudgmentsProcessor.JudgmentsProcessor(judgments, signatures, 'taggedRank.csv',
                                                        'filteredRank.csv')

In [6]:
X_tagged, Y_tagged, X_non_tagged, Y_non_tagged, judgments_categories_counter = judge_processor.process_judgments()

  0%|          | 0/777 [00:00<?, ?it/s]

Before tagging:777


100%|██████████| 777/777 [39:27<00:00,  3.05s/it]


After tagging:777
After tagging:777
Saving judgments to files
['w', 'z', 'na', 'do', 'rok', 'być', 'dzień', 'on', 'sąd', 'praca', '1', 'ten', 'od', 'przez', 'który', 'o', 'art', 'k', 'sprawa', 'złoty']
['na', 'do', 'nie', 'że', 'dnia', 'się', 'art', 'od', 'przez', 'sąd', 'roku', 'pracy', 'za', 'jest', 'oraz', 'zł', 'tym', 'to', 'sądu', 'co']


In [27]:
print (len(X_tagged))
print (len(X_non_tagged))
print (len(Y_non_tagged))
print (len(Y_tagged))

777
777
777
777


In [21]:
pprint.pprint (judgments_categories_counter)

Counter({'sprawy z zakresu ubezpieczenia społecznego': 227,
         'sprawy cywilne': 197,
         'sprawy karne': 126,
         'sprawy gospodarcze': 95,
         'sprawy w zakresie prawa pracy': 63,
         'sprawy w zakresie prawa rodzinnego': 39,
         'sprawy o wykroczenia': 30})


In [28]:
judgment_classifier = JudgmentsClassifier.JudgmentsClassifier(X_tagged, Y_tagged)
result_dict, clf, hyper_params = judgment_classifier.transform_and_train_classifier()

/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 8
  .format(len(labels), len(target_names))


In [29]:
print (hyper_params)
print(result_dict['accuracy_score'])
print(result_dict['classification_report'])
print(result_dict['micro_report'])
print(result_dict['macro_report'])

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.912820512821
                                            precision    recall  f1-score   support

                            sprawy cywilne       0.84      0.87      0.85        54
sprawy z zakresu ubezpieczenia społecznego       1.00      1.00      1.00        57
                              sprawy karne       1.00      0.93      0.97        30
                        sprawy gospodarcze       0.65      0.87      0.74        15
             sprawy w zakresie prawa pracy       1.00      0.71      0.83        14
        sprawy w zakresie prawa rodzinnego       1.00      0.93      0.96        14
                      sprawy o wykroczenia       0.91      0.91      0.91        11

                               avg / total       0.92      0.91      0.91       195

(0.9128205128205128, 0.9128205128205128, 0.9128205128205128, None)
(0.91405380333951758, 0.88890263175977469, 0.89547243488130179, None)


In [30]:
judgment_classifier = JudgmentsClassifier.JudgmentsClassifier(X_non_tagged, Y_non_tagged)
result_dict, clf, hyper_params = judgment_classifier.transform_and_train_classifier()

/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 8
  .format(len(labels), len(target_names))


In [31]:
print (hyper_params)
print(result_dict['accuracy_score'])
print(result_dict['classification_report'])
print(result_dict['micro_report'])
print(result_dict['macro_report'])

{'C': 15000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.902564102564
                                            precision    recall  f1-score   support

                            sprawy cywilne       0.83      0.89      0.86        54
sprawy z zakresu ubezpieczenia społecznego       1.00      1.00      1.00        57
                              sprawy karne       0.93      0.93      0.93        30
                        sprawy gospodarcze       0.67      0.80      0.73        15
             sprawy w zakresie prawa pracy       1.00      0.71      0.83        14
        sprawy w zakresie prawa rodzinnego       1.00      0.93      0.96        14
                      sprawy o wykroczenia       0.89      0.73      0.80        11

                               avg / total       0.91      0.90      0.90       195

(0.90256410256410258, 0.90256410256410258, 0.90256410256410269, None)
(0.90235358511220576, 0.85605029890744178, 0.87343503057788763, None)
